In [18]:
import elevenlabs
import requests
import os
import json
import base64
import pygame
import time

In [2]:
text=input('Enter your story')

In [19]:
from unrealspeech import UnrealSpeechAPI

speech_api = UnrealSpeechAPI(api_key=os.getenv('UNREAL_SPEACH_API_KEY'))


response = speech_api.speech(
    text='''The rain tapped gently against the window as Arjun sat in the dim café, sketching faces he’d never met.
A woman in a red coat walked in, her eyes scanning the room like she was searching for something… or someone.
She spotted Arjun, smiled faintly, and sat across from him without a word.
He blinked, confused.
“You draw strangers,” she said softly.
“Yes… how do you know?”
“Because you’ve drawn me before,” she whispered, sliding a worn notebook onto the table.
Inside, every page was filled with his sketches—years before they ever met.''',
    voice_id="Will",      # Choose voice: Scarlett, Dan, Liv, etc.
    timestamp_type="sentence",    # word or sentence
    bitrate="192k"
)




In [20]:
response

{'CreationTime': '2025-08-14T02:53:34.30Z',
 'OutputUri': 'https://unreal-expire-in-90-days.s3-us-west-2.amazonaws.com/e9267cb8-eb28-46a7-89b9-978f3358a667-0.mp3',
 'RequestCharacters': 538,
 'TaskId': 'e9267cb8-eb28-46a7-89b9-978f3358a667',
 'TaskStatus': 'completed',
 'TimestampsUri': 'https://unreal-expire-in-90-days.s3-us-west-2.amazonaws.com/e9267cb8-eb28-46a7-89b9-978f3358a667-0.json',
 'VoiceId': 'Will'}

In [6]:
sound=requests.get(response['OutputUri'])
timestampurl=requests.get(response['TimestampsUri'])

In [8]:
with open('timestamps.json','wb') as file:
    file.write(timestampurl.content)


with open('test.mp3','wb') as file:
    file.write(sound.content)

In [9]:
data_str = (timestampurl.content).decode("utf-8")  # bytes → string
data_json = json.loads(data_str) 

In [10]:
data_json

[{'start': 0.5, 'end': 0.5773998498916626, 'word': 'The', 'text_offset': 0},
 {'start': 0.5773998498916626,
  'end': 0.9489191224177679,
  'word': 'rain',
  'text_offset': 4},
 {'start': 0.9489191224177679,
  'end': 1.3552683368325236,
  'word': 'tapped',
  'text_offset': 9},
 {'start': 1.3552683368325236,
  'end': 1.7267876192927363,
  'word': 'gently',
  'text_offset': 16},
 {'start': 1.7267876192927363,
  'end': 2.1253968390325704,
  'word': 'against',
  'text_offset': 23},
 {'start': 2.1253968390325704,
  'end': 2.2221466513971486,
  'word': 'the',
  'text_offset': 31},
 {'start': 2.2221466513971486,
  'end': 2.7097656913101673,
  'word': 'window',
  'text_offset': 35},
 {'start': 2.7097656913101673,
  'end': 3.1548148207366467,
  'word': 'as',
  'text_offset': 42},
 {'start': 3.1548148207366467,
  'end': 3.55729403719306,
  'word': 'Arjun',
  'text_offset': 45},
 {'start': 3.55729403719306,
  'end': 3.9326833014686904,
  'word': 'sat',
  'text_offset': 51},
 {'start': 3.9326833014

In [12]:
import ffmpeg

def group_words_into_phrases(words, max_words_per_phrase=3):
    """Groups individual words (dict format) into phrases based on punctuation or max word count."""
    phrases = []
    current_phrase = []
    if not words:
        return []

    phrase_start_time = words[0]["start"]

    for i, w in enumerate(words):
        word = w["word"]
        start = w["start"]
        end = w["end"]

        current_phrase.append(word)
        should_end_phrase = (
            any(c in word for c in ",.?!")
            or len(current_phrase) >= max_words_per_phrase
            or i == len(words) - 1
        )

        if should_end_phrase:
            phrase_text = " ".join(current_phrase)
            phrases.append({
                "phrase": phrase_text,
                "start": phrase_start_time,
                "end": end
            })
            current_phrase = []
            if i < len(words) - 1:
                phrase_start_time = words[i + 1]["start"]

    return phrases



def add_timed_text(stream, text, start, end, fontsize_ratio=0.12, fontcolor="white", outline=6):
    """
    Adds large, bold, centered YouTube Shorts-style text.
    """
    return stream.drawtext(
        text=text,
        fontfile=r"C:\Windows\Fonts\impact.ttf",   # Bold meme/viral look
        fontsize=f"h*{fontsize_ratio}",            # Scale with video height
        fontcolor=fontcolor,
        x="(w-text_w)/2",                          # Center horizontally
        y="(h-text_h)/2",                          # Center vertically
        borderw=outline,                           # Thick outline
        bordercolor="black",
        box=0,                                     # No box, outline only
        enable=f"between(t,{start},{end})"
    )

def overlay(stream, phrases):
    """
    Apply timed phrases to the video stream.
    phrases: list of dicts with keys 'phrase', 'start', 'end'
    """
    for p in phrases:
        stream = add_timed_text(stream, p["phrase"], p["start"], p["end"])
    return stream


In [13]:
phrases=group_words_into_phrases(data_json)

In [26]:
data_json[len(data_json)-1]['end']

40.0531216869292

In [ ]:
input_file = "video.mp4"
audio_file = "test.mp3"
output_file = "asscream.mp4"

audio_duration=data_json[len(data_json)-1]['end']+1


video = ffmpeg.input(input_file,stream_loop=-1,t=audio_duration).video
audio = ffmpeg.input(audio_file).audio

video = overlay(video, phrases)

(
    ffmpeg
    .output(video, audio, output_file, vcodec='libx264', acodec='aac', strict='experimental')
    .overwrite_output()
    .run()
)


(None, None)